In [3]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import string

In [5]:
data = pd.read_csv('SMSSpamCollection.txt',sep='\t',names=['label','message'])

In [6]:
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
import re
ps = PorterStemmer()

In [10]:
corpus = []
for i in range(0,len(data)):
    a = re.sub('[a-zA-A]',' ',data['message'][i])
    a = a.lower()
    a = a.split()
    a = [ps.stem(word) for word in a if word not in stopwords.words('english')]
    a = ' '.join(a)
    corpus.append(a)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [12]:
X = cv.fit_transform(corpus).toarray()

In [13]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
y = pd.get_dummies(data['label'])

In [20]:
y

,ham,spam
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False
...,...,...
5567,False,True
5568,True,False
5569,True,False
5570,True,False


In [21]:
y = y.iloc[:,1].values

In [24]:
type(y)

numpy.ndarray

In [27]:
y

,0
0,False
1,False
2,True
3,False
4,False
...,...
5567,True
5568,False
5569,False
5570,False


In [29]:
def col(y):
    if y==True:
        return 1
    else:
        return 0
y[0] = y[0].apply(col)

In [30]:
y

,0
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [41]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=45)

In [42]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [43]:
nb.score(x_train,y_train)

0.9725641025641025

In [44]:
from sklearn.metrics import accuracy_score
y_pred = nb.predict(x_test)

In [45]:
print(accuracy_score(y_pred,y_test))

0.9557416267942583


In [46]:
import pickle
with open('spammodel.sav','wb') as f:
    pickle.dump(nb,f)